In [16]:
import pandas as pd

In [17]:
m = pd.read_csv(
    '../data/ml-1m/movies.dat',
    sep="::",
    names=["movieId", "title", "genres"],
    encoding='latin-1',
    engine='python'
)
m = m[m.movieId.notnull()].reindex()
m['itemid']=m.movieId.apply(lambda x: str(int(x)))
m['product_name'] = m['title']
items = m[['itemid','product_name','genres']]
# items.to_json('items.json')
items

,itemid,product_name,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [18]:
items.to_json('../data/ml-1m/processed/items.json')

### Purchases and grouped purchases.

In [19]:
interactions = pd.read_csv(
    '../data/ml-1m/ratings.dat',
    sep="::",
    names=["userId", "movieId", "rating", "timestamp"],
    encoding='latin-1',
    engine='python'
)
interactions = interactions[interactions.rating>=4.0]
interactions = interactions.sort_values(['userId','timestamp']) 
interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 575281 entries, 31 to 1000042
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   userId     575281 non-null  int64
 1   movieId    575281 non-null  int64
 2   rating     575281 non-null  int64
 3   timestamp  575281 non-null  int64
dtypes: int64(4)
memory usage: 21.9 MB


In [20]:
interactions['itemid'] = interactions['movieId'].apply(str)
interactions['userid'] = interactions['userId'].apply(str)
interactions['amount'] = 1
interactions['date'] = interactions['timestamp']
interactions[['itemid', 'userid','amount','date']]
interactions

,userId,movieId,rating,timestamp,itemid,userid,amount,date
31,1,3186,4,978300019,3186,1,1,978300019
22,1,1270,5,978300055,1270,1,1,978300055
27,1,1721,4,978300055,1721,1,1,978300055
37,1,1022,5,978300055,1022,1,1,978300055
36,1,1836,5,978300172,1836,1,1,978300172
...,...,...,...,...,...,...,...,...
1000119,6040,3671,4,997454367,3671,6040,1,997454367
999923,6040,232,5,997454398,232,6040,1,997454398
1000019,6040,2917,4,997454429,2917,6040,1,997454429
999988,6040,1921,4,997454464,1921,6040,1,997454464


In [21]:
interactions.to_json('../data/ml-1m/processed/purchases.json')

In [22]:
interactions['itemids'] = interactions[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x : ','.join(x))
iii = interactions[['userId','itemids']].drop_duplicates()
iii

,userId,itemids
31,1,"3186,1270,1721,1022,1836,3408,2804,1207,1193,2..."
130,2,"1198,1210,1293,2943,1225,1193,318,3030,2858,19..."
202,3,"2858,1968,1961,1266,1378,1379,3671,590,260,119..."
244,4,"1097,3468,480,260,1198,1387,2028,2366,1201,269..."
378,5,"908,919,1250,2858,2997,2770,2355,2908,3016,259..."
...,...,...
999375,6036,"1721,1883,2858,2997,2710,223,3203,1093,3408,25..."
999658,6037,"1267,2028,3148,858,562,912,1221,923,2804,2858,..."
999735,6038,"1210,2146,356,1079,1148,3548,3088,232,1136,122..."
999794,6039,"282,111,2067,930,3022,3088,2804,955,1276,2791,..."


In [23]:
iii.to_json('../data/ml-1m/processed/purchases_txt.json')

In [24]:
purchases=pd.read_json('../data/ml-1m/processed/purchases.json')
purchases['userid'] = purchases.userid.apply(str)
purchases['itemid'] = purchases.itemid.apply(str)
purchases_item_counts = purchases[['userid','itemid']]
purchases_user_counts = purchases[['userid','itemid']]
purchases_user_count = purchases.groupby(['userid']).size().to_frame('nr_of_purchases').reset_index()
purchases_user_count = purchases_user_count.sort_values(by=['nr_of_purchases'], ascending=False)
pu5=purchases_user_count[purchases_user_count.nr_of_purchases>=5]
purchases_pu5 = purchases[purchases.userid.isin(pu5.userid)]
purchases_item_count_pu5 = purchases_pu5.groupby(['itemid']).size().to_frame('nr_of_purchases').reset_index()
purchases_item_count_pu5 = purchases_item_count_pu5.sort_values(by=['nr_of_purchases'], ascending=False)


In [25]:
purchases_pu5.to_json('../data/ml-1m/processed/purchases_pu5.json')

In [26]:
purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = purchases_pu5[['userId','itemids']].drop_duplicates()
iii['userid']=iii['userId'].apply(str)
iii = iii[['userid','itemids']]


<ipython-input-26-dca95e343ced>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))


In [27]:
iii.to_json('../data/ml-1m/processed/purchases_txt_pu5.json')

In [28]:
iii['userid'].to_frame().to_json('../data/ml-1m/processed/users_pu5.json')

In [29]:

items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("../data/ml-1m/processed/items_pu5.json")

In [30]:
purchases_item_count_pu5.to_json("../data/ml-1m/processed/items_sorted_pu5.json")

In [31]:

pu5.to_json("../data/ml-1m/processed/users_sorted_pu5.json")

In [32]:

users = pd.read_json('../data/ml-1m/processed/users_pu5.json')
shuffled_users = users.sample(frac=1., random_state=42)
test_users = shuffled_users.iloc[:1200]
val_users = shuffled_users.iloc[1200:2400]
train_users = shuffled_users.iloc[2400:]

test_users.to_json("../data/ml-1m/processed/test_users.json")
val_users.to_json("../data/ml-1m/processed/val_users.json")
train_users.to_json("../data/ml-1m/processed/train_users.json") 
print("Total number of users in pu5", len(users))
len(train_users),len(val_users),len(test_users)

Total number of users in pu5 6034


(3634, 1200, 1200)

In [33]:
!ls ../data/ml-1m/processed/*.json

../data/ml-1m/processed/items.json
../data/ml-1m/processed/items_pu5.json
../data/ml-1m/processed/items_sorted_pu5.json
../data/ml-1m/processed/purchases.json
../data/ml-1m/processed/purchases_pu5.json
../data/ml-1m/processed/purchases_txt.json
../data/ml-1m/processed/purchases_txt_pu5.json
../data/ml-1m/processed/test_users.json
../data/ml-1m/processed/train_users.json
../data/ml-1m/processed/users_pu5.json
../data/ml-1m/processed/users_sorted_pu5.json
../data/ml-1m/processed/val_users.json
